In [1]:
## Qt-1, Qt-2, Pt-1, T(max,t-1),  
#Reading /Users/mezzatab/Dropbox/Hydrology/Data/Data/hydrology.nws.noaa.gov/pub/gcip/mopex/US_Data/Us_438_Daily ## 
# It has Q,P, evaporation, tmax, tmin #
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import sklearn
print sklearn.__version__
from sklearn.model_selection import ShuffleSplit, train_test_split
from CastFloatCastInt import castFloat, castInt 

Path="/Users/mezzatab/Dropbox/Hydrology/Data/Data/hydrology.nws.noaa.gov/pub/gcip/mopex/US_Data/Us_438_Daily/"
PathQ="/Users/mezzatab/Dropbox/Hydrology/Data/Data/hydrology.nws.noaa.gov/pub/gcip/mopex/US_Data/Daily Q 1800/"
ImagePath="/Users/mezzatab/Dropbox/Hydrology-ML/Images/"

0.19.1


In [2]:
## Check specfic area : 03451500.dly # 
## Reading Data ## 
pathFile=Path+"03451500.dly"
pathQFile=PathQ+"03451500.dq"
startYear=1948 ## 

f=open(pathFile) 
data=f.readlines()

fQ=open(pathQFile) 
dataQ=fQ.readlines()

In [3]:
#dataFrame=pd.DataFrame()#columns=["Time","precipitation","evaporation","discharge","Tmax","Tmin"])
rows=0
x=0
Time=[]
Prec=[]
Eva=[]
Discharge=[]
Tmax=[]
Tav=[]
Tmin=[]
for line in data:
    List=line.split()
    Row=[]
    L=len(List)
    if L == 8: 
        time=List[0]+'0'+List[1]+'0'+List[2]
        x=3
    if L == 7:
        time=List[0]+'0'+List[1]#+List[2]
#        print time
        x=2
    if L == 6:
        time=List[0]
        x=1
    year=castInt(time)/10000
    if year >= startYear:
        Time.append(castInt(time))
        Prec.append(castFloat(List[x]))
        Eva.append(castFloat(List[x+1]))
        Discharge.append(castFloat(List[x+2]))
        Tmax.append(castFloat(List[x+3]))
        Tmin.append(castFloat(List[x+4]))
        Tav.append((castFloat(List[x+3])+castFloat(List[x+4]))/2.0)
    rows=rows+1
#dataFrame=pd.DataFrame({'Time':Time,'P':Prec, 'Eva':Eva, 'Discharge':Discharge,'T':Tav, 'Tmax':Tmax,'Tmin':Tmin })
#dataFrame=pd.DataFrame({'Time':Time,'P':Prec, 'Eva':Eva, 'Q':Discharge,'Tmax':Tmax,'Tmin':Tmin })

## Reading Q data from Different file ## 
File="/Users/mezzatab/Dropbox/Hydrology/Data/Data/hydrology.nws.noaa.gov/pub/gcip/mopex/US_Data/Daily Q 1800/"

In [4]:
## Reading Q data from Different file ## 
qTime=[]
Q=[]
for line in dataQ[1:]:
#    month=[]
    Month=line.split()
    year=castInt(Month[0])/100
    month=castInt(Month[0])-100*year
    if year >= startYear:     
        for i,t in enumerate(Month[1:]):
            try:
                if castFloat(t) > 0:
                    Q.append(castFloat(t))
                    if i < 9:
                        time=Month[0]+'0'+str(i+1)
                    else:
                        time=Month[0]+str(i+1)
                    qTime.append(time)
            except ValueError:
                pass

In [5]:
# The last data for Q is 2002-09-30, which is 19997 elements or Q[19996]
dataFrame=pd.DataFrame({'Time':Time[:19997],'Q':Q[:19997], 'P':Prec[:19997], 'PET':Eva[:19997],'Discharge':Discharge[:19997],'T':Tav[:19997],'Tmax':Tmax[:19997],'Tmin':Tmin[:19997] })
#print len(Q)
#print len(Time[:19997])

## tranforming dataframe from daily to monthly average ## 

In [6]:
line=dataFrame.loc[0,"Time"]
monthOld=int(line/100)-100*(int(line/10000))
mFrac=(monthOld-1)/12.0
year=int(line/10000)+mFrac
time=int(line/100)
qav=0
Tmin_av=0
Tmax_av=0
Ev_av=0
Pav=0
T_av=0
count=0

Time=[]
Prec=[]
Eva=[]
Discharge=[]
Tmax=[]
Tav=[]
Tmin=[]
for i in range(len(dataFrame.loc[:,"Time"])):
    line=dataFrame.loc[i,"Time"]
    month=int(line/100)-100*(int(line/10000))

    if month==monthOld:
        count=count+1
        qav+=dataFrame.loc[i,"Q"]
        Pav+=dataFrame.loc[i,"P"]
        T_av+=dataFrame.loc[i,"T"]
        Tmin_av+=dataFrame.loc[i,"Tmin"]
        Tmax_av+=dataFrame.loc[i,"Tmax"]
        Ev_av+=dataFrame.loc[i,"PET"]
    else:
        qav/=count
#        Pav/=count
        T_av/=count
        Tmin_av/=count
        Tmax_av/=count
        Ev_av/=count

        Time.append(year)
        time=int(line/100)    
        Prec.append(Pav)     
        Tav.append(T_av)
        Eva.append(Ev_av)
        Tmax.append(Tmax_av)
        Tmin.append(Tmin_av)
        Discharge.append(qav)
        mFrac=(month-1)/12.0
        year=int(line/10000)+mFrac

        
        count=0
        monthOld=month
        qav=0
        Pav=0
        T_av=0
        Tmin_av=0
        Tmax_av=0
        Ev_av=0
MonthFrame=pd.DataFrame({'Time':Time,'P':Prec, 'PET':Eva, 'Q':Discharge,'T':Tav, 'Tmax':Tmax,'Tmin':Tmin })

## Reading the new set of Data ## 2004-2017

In [7]:
## Reading New Years Data ## 
Path_NewData="/Users/mezzatab/Dropbox/Hydrology/Data/Data/Codes/New_Data/"
P_NY="Precipitation_2014-2018" 
Q_NY="Discharge_2004-2018"
T_NY="All_T.csv"

In [8]:
PathP=Path_NewData+P_NY
PathQ=Path_NewData+Q_NY
PathT=Path_NewData+T_NY
fN_P=open(PathP) 
dataN_P=fN_P.readlines()

fN_Q=open(PathQ) 
dataN_Q=fN_Q.readlines()

fN_T=open(PathT) 
dataN_T=fN_T.readlines()